In [1]:
from pyspark.sql.types import *
from pyspark.sql import Row
import ast
import json
import pandas as pd
import numpy as np
import requests
import pickle
from datetime import datetime
import matplotlib.pyplot as plt
import os
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql import HiveContext
hiveContext=HiveContext(sc)
import sys
from numpy import *
#%matplotlib inline
dbutils.fs.mount("s3n://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, 'data-analysis-sachin'), "/mnt/%s" % assignment1)
dbutils.fs.ls('/mnt/assignment1/csvParquetFiles_new/')

In [2]:
#Model Classes
# %load PieceWise.py
#Encoder class for spark dataframes
import pandas as pd
from numpy import *
class encoder:
    """
    The encoder/decoder class is the base class for all encoder/decoder pairs.
    Subclasses encode different types of encoding.
    EncoderLearner is a factory class for fitting encoders to data
    """
    def __init__(self,raw,max_gap):
        """
        given a spark DataFrame or Series (raw), find the best model of a given type
        """
    
    def compress(self):
        """
        given a raw sequence and a model, return a compressed representation.
        """
        self.compressed=None
        return self.compressed
    
    def recon(self,compressed):
        """
        Recreate the original DataFrame or Series, possibly with errors.
        """
        Recon=None
        return Recon
    
    def get_size(self):
        return len(self.compressed)
    
    def compute_error(self,S,compressed):
        if type(compressed)==type(None):
            compressed=self.compressed
        #R=self.recon(compressed=compressed,index=S.index)
        R=self.recon()
        V=R-S
        V=V.dropna()
        return sqrt(sum([v*v for v in V.values]))/len(V)
      
class piecewise_constant(encoder):
    """ 
    Represent the signal using a sequence of piecewise constant functions 
    """
    def __init__(self,raw,max_gap):
      S=raw
      if type(S) != pd.Series:
        raise 'encode expects pandas Series as input'
      self.index=S.index
      self.Sol=self.fit(S,max_gap)
    
    # fit uses dynamic programming to find the best piecewise constant solution
    # max_gap is the maximal extent of a single step.
    # Reason for max_gap is that even if the error is small we want to correct
    # it with some minimal frequence. 
    # Not quite a snapshot because the value will not necessarily change after 
    # max_gap is reached.
    def fit(self,S,max_gap):
        S[np.isnan(S)]=0
        _range=np.max(S)-np.min(S)
        # _range is a constant that is added to the error at each stop point
        # Larger values will cause fewer switches.
        print 'range=',_range
        #Dynamic programming
        Sol=[[]]*len(S)  # an array that holds the best partition ending at each point of the sequence.
                # Each element contains a best current value, a pointer to the last change in best 
                # solution so far and the total error of best solution so far.
        for i in range(len(S)):
            if i==0:
                Sol[i]={'prev':None, 'value':S[0], 'error':0.0, 'switch_no':0}
            # Sol is indexed by the location in the sequence S
            # prev: the index of the last switch point
            # value: current prediction value
            # error: cumulative error to this point
            # switch_no: number of switches so far.
            else:
                err0 = Sol[i-1]['error']+(Sol[i-1]['value']-S[i])**2
                best=None
                best_err=1e20
                best_val=S[i]
                for j in xrange(np.max([0,i-max_gap]),i):
                      _mean=np.mean(S[j:i])
                      _std=np.std(S[j:i])
                      err=_std*(i-j)+Sol[j]['error']+_range
                      if err<best_err:
                          best=j
                          best_val=_mean
                          best_err=err
                Sol[i]={'prev':best, 'value':best_val, 'error':best_err,\
                        'switch_no': Sol[best]['switch_no']+1}
            #print '\r',i,Sol[i],
        return Sol
    
    def compress(self,S):
        Switch_points=[]
        i=len(self.Sol)-1                # start from the end 
        while i>0:
            prev=self.Sol[i]['prev']
            value=self.Sol[i]['value']
            if self.Sol[prev]['value'] != value:
                Switch_points.append({'time':S.index[prev],'value':value})
            i=prev
        self.compressed=Switch_points
        return Switch_points

    def recon(self,compressed=None, index=None):
        #print '\nindex=',index==None,'\n'
        #print '\ncompressed=',compressed==None,'\n'
        if(type(index)==type(None)):
            index=self.index
        Recon=pd.Series(index=index)
        
        if(type(compressed)==type(None)):
            compressed=self.compressed
        for e in compressed:
            time=e['time']
            value=e['value']
            Recon[time]=value
            
        return Recon.fillna(method='ffill')
      
class piecewise_linear(encoder):
    """ 
    Represent the signal using a sequence of piecewise linear functions 
    """
    def __init__(self,raw,max_gap):
      S=raw
      if type(raw) != pd.Series:
        raise 'encode expects pandas Series as input'
      self.index=raw.index
      self.Sol=self.fit(raw,max_gap)
    
    # fit uses dynamic programming to find the best piecewise linear solution
    # max_gap is the maximal extent of a single step.
    # Reason for max_gap is that even if the error is small we want to correct
    # it with some minimal frequence. 
    # Not quite a snapshot because the value will not necessarily change after 
    # max_gap is reached.
    def fit(self,S,max_gap):
        S[np.isnan(S)]=0
        _range=np.max(S)-np.min(S)
        # _range is a constant that is added to the error at each stop point
        # Larger values will cause fewer switches.
        print 'range=',_range
        #Dynamic programming
        Sol=[[]]*len(S)  # an array that holds the best partition ending at each point of the sequence.
                # Each element contains a best current value, a pointer to the last change in best 
                # solution so far and the total error of best solution so far.
        for i in range(len(S)):
            if i==0:
                Sol[i]={'prev':None, 'value':S[0], 'error':0.0, 'switch_no':0, 'slope':0}
            # Sol is indexed by the location in the sequence S
            # prev: the index of the last switch point
            # value: current prediction value
            # error: cumulative error to this point
            # switch_no: number of switches so far.
            #slope: slope of th linear line at this point
            else:
                err0 = Sol[i-1]['error']+(Sol[i-1]['value']-S[i])**2
                best=None
                best_err=1e20
                best_val=S[i]
                best_slope=1e20
                for j in xrange(np.max([0,i-max_gap]),i):
                    #_mean=np.mean(S[j:i])
                    _slope=(S[i]-S[j])*1.0/(i-j)
                    #_std=np.std(S[j:i]
                    _val=0
                    _err=0
                    for k in xrange(j,i):
                      _val=Sol[j]['value']+_slope*(k-j)
                      _err+=(Sol[k]['value']-_val)**2
                    err=_err*1.0/(i-j)+Sol[j]['error']+_range
                    _val=Sol[j]['value']+_slope*(i-j)
                    if err<best_err:
                        best=j
                        best_val=_val
                        best_err=err
                        best_slope=_slope
                Sol[i]={'prev':best, 'value':best_val, 'error':best_err,\
                        'switch_no': Sol[best]['switch_no']+1, 'slope':best_slope}
            #print '\r',i,Sol[i],
        return Sol
    
    def compress(self,S):
        Switch_points=[]
        i=len(self.Sol)-1                # start from the end 
        while i>0:
            prev=self.Sol[i]['prev']
            slope=self.Sol[i]['slope']
            value=self.Sol[i]['value']
            if self.Sol[prev]['slope'] != slope:
                Switch_points.append({'time':S.index[prev],'value':value})
            i=prev
        self.compressed=Switch_points
        return Switch_points

    def recon(self,compressed=None, index=None):
        #print '\nindex=',index==None,'\n'
        #print '\ncompressed=',compressed==None,'\n'
        print "HI"
        if(type(index)==type(None)):
            index=self.index
        Recon=pd.Series(index=index)
        if(type(compressed)==type(None)):
            compressed=self.compressed
        for e in compressed:
            time=e['time']
            value=e['value']
            Recon[time]=value
        
        #print Recon    
        #return Recon.fillna(method='ffill')
        Recon.interpolate(method="linear", inplace=True)
        return Recon

In [3]:
#All applicable functions
def runAnalysisOld(room,template,stTime,enTime,method,tolerance):
  dataDF=loadf(room,template,stTime,enTime)
  print len(dataDF)
  [compressedDF, reconDF]=model(dataDF,method,tolerance)
  return [dataDF, compressedDF, reconDF]
  
def loadf(room,template,stTime,enTime):
  query = "select * from allbuildingsensordata where room='"+str(room)+"' and template='"+str(template)+"' and timeseries between '"+ str(stTime) +"' and '"+str(enTime)+"'"
  #print query
  df=hiveContext.sql(query)
  dataDF=pd.DataFrame(df.select('timeseries','values').collect(),columns=['timeseries','values'])
  dataDF['timeseries']=dataDF['timeseries'].apply(lambda x:datetime.strptime(x, '%Y-%m-%dT%H:%M:%S+00:00'))
  dataDF['values']=dataDF['values'].apply(lambda x:float(str(x)))
  return dataDF
    
def dataPlot(dfArgument,timeLowerBound, timUpperBound):
  tempDF=dfArgument[dfArgument.timeseries.between(timeLowerBound, timUpperBound)]
  #if(Rooms!='All'):
  #  tempDF=tempDF[tempDF['room']==Rooms]
  #display(tempDF)
  dataDF=pd.DataFrame(tempDF.select('timeseries','values').limit(5000).collect(),columns=['timeseries','values'])
  return dataDF

def model(A,method,tolerance):  
  pd_df=A
  sys.stdout.flush()
  #pd_df=pd.DataFrame(A.select('timeseries','values').limit(5000).collect(),columns=['timeseries','values'])
  #pd_df.plot(kind='line')
  S=pd_df['values']
  _std=np.std(S)
  print "Std dev is ",_std
  if(method=='piecewise_constant'):
    encoder=piecewise_constant(S,tolerance)
  elif(method=='piecewise_linear'):
    encoder=piecewise_linear(S,tolerance)
  C=encoder.compress(S)
  R=encoder.recon()
  print "type is ",type(R)
  compressed_df=pd.DataFrame(C)
  print 'size=',encoder.get_size(),
  error=encoder.compute_error(S,compressed=C)
  print 'error=',error, 'error/_std=',error/_std
  print C
  return [compressed_df, R]
  
def runAnalysis(table,stTime,enTime,templateCount):
  templates=['Zone Temperature','Actual Supply Flow','Occupied Clg Min','Occupied Htg Flow','Common Setpoint', 'Actual Heating Setpoint', 'Supply Vel Press', 'Zone Temperature Error',  'Damper Position',  'Warm Cool Adjust', 'Cooling Command', 'HVAC Zone Power', 'Damper Command', 'Cooling Max Flow', 'Occupied Htg Flow','Actual Cooling Setpoint', 'Reheat Valve Command Error']
  dfs=[]
  plotTemplates=[]
  for t in templates[0:templateCount]:
    try:
      query = "select * from "+str(table)+" where template='"+str(t)+"' and timeseries between '"+ str(stTime) +"' and '"+str(enTime)+"'" 
      print query
      df=hiveContext.sql(query)
      dataDF=pd.DataFrame(df.select('timeseries','values').collect(),columns=['timeseries','values'])
      dataDF['timeseries']=dataDF['timeseries'].apply(lambda x:datetime.strptime(x, '%Y-%m-%dT%H:%M:%S+00:00'))
      dataDF['values']=dataDF['values'].apply(lambda x:float(str(x)))
      if(t in ['Zone Temperature','Zone Temperature Error']):
        method='piecewise_linear'
      else:
        method='piecewise_constant'
      [compressedDF, reconDF]=model(dataDF,method,tolerance=96)
      plotTemplates.append(t)
      dfs.extend([dataDF,reconDF]) 
    except:
      print t
  return [dfs,plotTemplates]
    
def getTime(x,dfTest):
  return dfTest.at[int(x),'timeseries']

def plotResults(dfs,plotTemplates):
  plt.title('compression analysis')
  #fig = plt.figure(figsize=(15, 8))
  fig, ax = plt.subplots(figsize=(15, 8))
  linestyles = ['_', '-', '--', ':']
  colors = ('b', 'g', 'r', 'c', 'm', 'y', 'k')
  timeX=dfs[0]['timeseries'].tolist()
  axes = [ax, ax.twinx()]
  axes1Count=0
  axes0Count=0
  for i in xrange(len(plotTemplates)):
    try:
      print 2*i
      ReconDF=dfs[2*i+1].to_frame(name='values')
      ReconDF['time']=ReconDF.index
      ReconDF=ReconDF.dropna()
      ReconDF=ReconDF.sort(['time'], ascending=[1])
      ReconDF['timeseries']=ReconDF.apply(lambda x: getTime(x['time'],dfs[2*i]), axis=1)
      if(plotTemplates[i] in ['Actual Supply Flow','Occupied Command','Damper Position']):
        axes1Count=1
        axes[1].plot(dfs[2*i]['timeseries'].tolist(),dfs[2*i]['values'].tolist(),'k--',color=colors[i],label=plotTemplates[i])
        axes[1].plot(ReconDF['timeseries'].tolist(),ReconDF['values'].tolist(),'k:',color=colors[i],label=plotTemplates[i]+'_reconstructed')
      else:
        axes0Count=1
        axes[0].plot(dfs[2*i]['timeseries'].tolist(),dfs[2*i]['values'].tolist(),'k--',color=colors[i],label=plotTemplates[i])
        axes[0].plot(ReconDF['timeseries'].tolist(),ReconDF['values'].tolist(),'k:',color=colors[i],label=plotTemplates[i]+'_reconstructed')
    except:
      print plotTemplates[i]
  if(axes1Count==0):
    print "axes1Count",axes1Count
    print len(timeX)
    axes[1].plot(timeX,[0 for x in timeX],'k',color='w',label='axes1')
  if(axes0Count==0):
    print "axes0Count",axes0Count
    axes[0].plot(timeX,[0 for x in timeX],'k',color='w',label='axes0')
  axes[0].legend(loc='upper left',fontsize='x-small')
  axes[1].legend(loc='upper right',fontsize='x-small')
  axes[1].set_ylabel('scale for top right legend')
  axes[0].set_ylabel('scale for top left legend')
  display(fig)
  
def loadTags():
  tagsDF=sqlContext.read.parquet("/mnt/assignment1/csvParquetFiles/tagsDF")
  keep=[tagsDF.name,tagsDF.sensor_id,tagsDF.template,tagsDF.timeseries_span]
  tagsDF=tagsDF.select(*keep)
  return tagsDF

In [4]:
dfs=runAnalysisOld('Rm-2219','Zone Temperature','2014-10-10T23:01:45+00:00','2014-10-24T23:59:18+00:00','piecewise_constant',tolerance=96)

In [5]:
[dfs,plotTemplates]=runAnalysis(table='rm4226',stTime='2013-12-01',enTime='2013-12-07',templateCount=2)
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [6]:
[dfs,plotTemplates]=runAnalysis(table='rm2138',stTime='2013-12-01',enTime='2013-12-07')
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [7]:
[dfs,plotTemplates]=runAnalysis(table='rm3214',stTime='2014-02-10',enTime='2014-02-20',templateCount=17)
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [8]:
[dfs,plotTemplates]=runAnalysis(table='rm2234',stTime='2014-01-20',enTime='2014-01-30',templateCount=17)
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [9]:
[dfs,plotTemplates]=runAnalysis(table='rm3208',stTime='2014-03-20',enTime='2014-03-30',templateCount=17)
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [10]:
[dfs,plotTemplates]=runAnalysis(table='rm3214',stTime='2014-02-10',enTime='2014-02-20',templateCount=17)
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [11]:
[dfs,plotTemplates]=runAnalysis(table='rm2138',stTime='2014-04-07',enTime='2014-04-14',templateCount=17)
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [12]:
[dfs,plotTemplates]=runAnalysis(table='rm2118',stTime='2013-07-01',enTime='2013-07-10',templateCount=17)
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [13]:
[dfs,plotTemplates]=runAnalysis(table='rm4226',stTime='2014-03-01',enTime='2014-03-07')
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [14]:
[dfs,plotTemplates]=runAnalysis(table='rm2118',stTime='2013-12-01',enTime='2013-12-10',templateCount=17)
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [15]:
[dfs,plotTemplates]=runAnalysis(table='rm2118',stTime='2014-06-01',enTime='2014-06-10',templateCount=17)
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [16]:
[dfs,plotTemplates]=runAnalysis(table='rm2118',stTime='2014-12-01',enTime='2014-12-10',templateCount=17)
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [17]:
[dfs,plotTemplates]=runAnalysis(table='rm2118',stTime='2015-04-01',enTime='2015-04-10',templateCount=17)
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [18]:
[dfs,plotTemplates]=runAnalysis(table='rm4226',stTime='2013-07-01',enTime='2013-07-07')
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [19]:
[dfs,plotTemplates]=runAnalysis(table='rm2138',stTime='2013-07-01',enTime='2013-07-07')
print len(dfs)
print plotTemplates
plotResults(dfs,plotTemplates)

In [20]:
#get summary table
df=hiveContext.sql("select * from dataSummary");
summaryDF=pd.DataFrame(df.collect(),columns=['room','template','stTime','enTime'])
summaryDF.head()

In [21]:
#df=loadf('RF2','Current','2013-07-01','2013-07-14')
df['timeseries']=df['timeseries'].apply(lambda x:datetime.strptime(x, '%Y-%m-%dT%H:%M:%S+00:00'))
df.head()

In [22]:
dfs=runAnalysis('RF2','Current','2013-07-01','2013-07-14','piecewise_constant',tolerance=16)
plotResults(dfs,'Current')

In [23]:
dfs=runAnalysisOld('RM-1125','Actual Heating Setpoint','2015-02-01T00:00:00+00:00','2015-02-14T00:00:00+00:00','piecewise_constant',tolerance=16)

In [24]:
dfs=runAnalysis('RM-1125','Actual Heating Setpoint','2015-02-01T00:00:00+00:00','2015-02-14T00:00:00+00:00','piecewise_constant',tolerance=16)
plotResults(dfs,'Actual Heating Setpoint')

In [25]:
dfs=runAnalysisOld('Rm-2219','Zone Temperature','2014-10-10T23:01:45+00:00','2014-10-24T23:59:18+00:00','piecewise_constant',tolerance=96)

In [26]:
dfs=runAnalysis('Rm-2219','Zone Temperature','2014-10-10T23:01:45+00:00','2014-10-24T23:59:18+00:00','piecewise_constant',tolerance=96)
plotResults(dfs,'Zone Temperature')

In [27]:
dfs=runAnalysisOld('Rm-2150','Actual Cooling Setpoint','2015-02-01T00:00:00+00:00','2015-02-14T00:00:00+00:00','piecewise_constant',tolerance=96)

In [28]:
dfs=runAnalysis('Rm-2150','Actual Cooling Setpoint','2015-02-01T00:00:00+00:00','2015-02-14T00:00:00+00:00','piecewise_constant',tolerance=96)
plotResults(dfs,'Actual Cooling Setpoint')

In [29]:
dfs=runAnalysis('Rm-2109','Warm Cool Adjust','2014-03-01T00:00:00+00:00','2014-03-14T00:00:00+00:00','piecewise_constant',tolerance=96)
plotResults(dfs,'Warm Cool Adjust')

In [30]:
dfs=runAnalysis('RM-1145','Common Setpoint','2014-12-01T00:00:00+00:00','2014-12-14T00:00:00+00:00','piecewise_constant',tolerance=36)
plotResults(dfs,'Common Setpoint')

In [31]:
dfs=runAnalysisOld('RM-B260','Damper Position','2014-12-01T00:00:00+00:00','2014-12-14T00:00:00+00:00','piecewise_constant',tolerance=36)

In [32]:
dfs=runAnalysis('RM-B260','Damper Position','2014-12-01T00:00:00+00:00','2014-12-14T00:00:00+00:00','piecewise_constant',tolerance=36)
plotResults(dfs,'Damper Position')

In [33]:
dfs=runAnalysisOld('RM-1106','Actual Sup Flow SP','2014-08-01T00:00:00+00:00','2014-08-14T00:00:00+00:00','piecewise_constant',tolerance=96)

In [34]:
dfs=runAnalysis('RM-1106','Actual Sup Flow SP','2014-08-01T00:00:00+00:00','2014-08-14T00:00:00+00:00','piecewise_constant',tolerance=96)
plotResults(dfs,'Actual Sup Flow SP')

In [35]:
dfs=runAnalysisOld('Rm-4127','Actual Supply Flow','2015-02-01T00:00:00+00:00','2015-02-14T00:00:00+00:00','piecewise_constant',tolerance=16)

In [36]:
dfs=runAnalysis('Rm-4127','Actual Supply Flow','2015-02-01T00:00:00+00:00','2015-02-14T00:00:00+00:00','piecewise_constant',tolerance=16)
plotResults(dfs,'Actual Supply Flow')

In [37]:
dfs=runAnalysis('Rm-3146','Supply Vel Press','2014-02-01T00:00:00+00:00','2014-02-14T00:00:00+00:00','piecewise_constant',tolerance=16)
plotResults(dfs,'Supply Vel Press')

In [38]:
dfs=runAnalysis('Rm-3213','Damper Command','2015-02-01T00:00:00+00:00','2015-02-14T00:00:00+00:00','piecewise_constant',tolerance=96)
plotResults(dfs,'Damper Command')

In [39]:
dfs=runAnalysis('Rm-4127','Actual Supply Flow','2015-02-01T00:00:00+00:00','2015-02-14T00:00:00+00:00','piecewise_constant',tolerance=96)
plotResults(dfs,'Actual Supply Flow')

In [40]:
#dfs=runAnalysis('HW-SYS','2015','2015-03-01 08:00:00','2015-03-08 17:30:00','piecewise_constant')
#df=loadf('HW-SYS','2015')
#subDF=dataPlot(df,'2015-03-01 08:00:00','2015-03-08 17:30:00')
#pd_df=subDF
#S=pd_df['values']
#_std=np.std(S)
#print "Std dev is ",_std

minErrorGap=10
minError=1e20
errors=[]
for maxGap in xrange(10,100):
  encoder=piecewise_constant(S,maxGap)
  C=encoder.compress(S)
  error=encoder.compute_error(S,C)
  print error
  errors.append(error)
  if(error<minError):
    minError=error
    minErrorGap=maxGap
print minErrorGap
minError
#R=encoder.recon()
#compressed_df=pd.DataFrame(C)


In [41]:
plt.plot(np.array(errors))
plt.title('errors vs max_gap')
display(fig)

In [42]:
sortedErrorIndex=sorted(range(len(errors)), key=lambda k: errors[k])
print sortedErrorIndex[0:10]

In [43]:
#Change variables here. general funtion signature is runAnalysis(room,year,startTime,endTime,method) where method can take 3 values-piecewise_constant, piecewise_linear, both. Return object is a list of dataframes in this order-spark dataframe of full data of the room for the given year. pandas dataframe of subset of above data within the given time range. pandas dataframe of the comressed data
dfs=runAnalysis('HW-SYS','2015','2015-03-01 08:00:00','2015-03-08 17:30:00','piecewise_constant',tolerance=36)
plotResults(dfs)

In [44]:
dfs=runAnalysis('HW-SYS','2015','2015-03-01 08:00:00','2015-03-08 17:30:00','piecewise_constant',tolerance=9)
plotResults(dfs)

In [45]:
dfs=runAnalysis('HW-SYS','2015','2015-03-01 08:00:00','2015-03-08 17:30:00','piecewise_linear',tolerance=9)
plotResults(dfs)

In [46]:
#Plots for other rooms, times and methods are below

In [47]:
dfs=runAnalysis('Rm-4126','2014','2014-12-01 08:00:00','2014-12-08 17:30:00','piecewise_constant',tolerance=96)
plotResults(dfs)

In [48]:
dfs=runAnalysis('Rm-2154','2014','2014-12-01 08:00:00','2014-12-08 17:30:00','piecewise_constant',tolerance=40)
plotResults(dfs)

In [49]:
dfs=runAnalysis('RM-1208B','2015','2015-03-01 08:00:00','2015-03-08 17:30:00','piecewise_constant',tolerance=96)
plotResults(dfs)

In [50]:
dfs=runAnalysis('Rm-3152','2015','2015-03-01 08:00:00','2015-03-08 17:30:00','piecewise_constant',tolerance=96)
plotResults(dfs)

In [51]:
dfs=runAnalysis('Rm-4130','2015','2015-03-01 08:00:00','2015-03-08 17:30:00','piecewise_constant',tolerance=96)
plotResults(dfs)